In [43]:
import zipfile
import os

kmz_file = "KMZ/todalared.kmz"
kml_file = "doc.kml"

os.chdir('c:/Users/artur/OneDrive/Documents/TrabajoTesis') 




In [44]:

kmz_file = "KMZ/todalared.kmz"

print(os.path.exists(kmz_file)) 


True


In [45]:
with zipfile.ZipFile(kmz_file, 'r') as kmz:
    print(kmz.namelist())

['doc.kml']


In [46]:
with zipfile.ZipFile(kmz_file, 'r') as kmz:
    kmz.extractall("KMZ_extracted")  # Extracts to a folder


In [47]:
from fastkml import kml

with open("KMZ/doc.kml", "rb") as f:
    kml_content = f.read()

k = kml.KML()
k.from_string(kml_content)

# Recursively print all placemarks and their geometry types
def print_features(features, level=0):
    for feature in features:
        indent = "  " * level
        print(f"{indent}Feature name: {feature.name}, type: {feature.__class__.__name__}")
        if hasattr(feature, 'geometry') and feature.geometry:
            print(f"{indent}  Geometry type: {feature.geometry.geom_type}")
        if hasattr(feature, 'features') and feature.features:
            print_features(feature.features, level + 1)

print_features(k.features)





In [48]:
import zipfile

kmz_file = "KMZ/todalared.kmz"

# Abrir KMZ y extraer KML
with zipfile.ZipFile(kmz_file, 'r') as kmz:
    kml_names = [name for name in kmz.namelist() if name.endswith('.kml')]
    kml_content = kmz.read(kml_names[0])

# Mostrar los primeros 2000 caracteres del KML para inspección
print(kml_content[:2000].decode('utf-8', errors='ignore'))


<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2" xmlns:kml="http://www.opengis.net/kml/2.2" xmlns:atom="http://www.w3.org/2005/Atom">
<Document>
	<name>todalared.kmz</name>
	<Style id="sn_E1703">
		<IconStyle>
			<scale>0.7</scale>
			<Icon>
				<href>http://maps.google.com/mapfiles/kml/paddle/E.png</href>
			</Icon>
			<hotSpot x="32" y="1" xunits="pixels" yunits="pixels"/>
		</IconStyle>
		<LabelStyle>
			<scale>0.7</scale>
		</LabelStyle>
		<ListStyle>
			<ItemIcon>
				<href>http://maps.google.com/mapfiles/kml/paddle/E-lv.png</href>
			</ItemIcon>
		</ListStyle>
		<LineStyle>
			<color>ff0000ff</color>
			<width>8</width>
		</LineStyle>
		<PolyStyle>
			<color>7f00ffff</color>
		</PolyStyle>
	</Style>
	<Style id="sh_E620">
		<IconStyle>
			<scale>0.827274</scale>
			<Icon>
				<href>http://maps.google.com/mapfiles/kml/paddle/E.png</href>
			</Icon>
			<hotSpot x="32" y="1" xunits="pixels" yunits="pixels"

In [49]:
from lxml import etree
import zipfile
import pandas as pd

# 1️⃣ Open KMZ and extract KML
kmz_file = "KMZ/todalared.kmz"
with zipfile.ZipFile(kmz_file, 'r') as kmz:
    kml_names = [name for name in kmz.namelist() if name.endswith('.kml')]
    kml_content = kmz.read(kml_names[0])

# 2️⃣ Parse XML
tree = etree.fromstring(kml_content)

# 3️⃣ Namespaces
ns = {
    "kml": "http://www.opengis.net/kml/2.2",
    "gx": "http://www.google.com/kml/ext/2.2"
}

stations = []

# 4️⃣ Traverse all Placemark elements
for placemark in tree.findall(".//kml:Placemark", namespaces=ns):
    name_elem = placemark.find("kml:name", namespaces=ns)
    name = name_elem.text if name_elem is not None else "Unknown"

    # Find all coordinates inside this Placemark
    for coord_elem in placemark.findall(".//kml:coordinates", namespaces=ns):
        coords_text = coord_elem.text.strip()
        for line in coords_text.split():
            parts = line.strip().replace(",", " ").split()
            if len(parts) >= 2:
                lon, lat = map(float, parts[:2])
                stations.append({
                    "name": name,
                    "longitude": lon,
                    "latitude": lat
                })

# 5️⃣ Save to CSV
df_stations = pd.DataFrame(stations)
df_stations.to_csv("estaciones.csv", index=False)
print(df_stations)





                                                    name  longitude   latitude
0      Red Troncal ALAMEDA (Santiago)-PUERTO (Valpara... -70.678720 -33.459852
1      Red Troncal ALAMEDA (Santiago)-PUERTO (Valpara... -70.678685 -33.458914
2      Red Troncal ALAMEDA (Santiago)-PUERTO (Valpara... -70.678677 -33.458612
3      Red Troncal ALAMEDA (Santiago)-PUERTO (Valpara... -70.678719 -33.458219
4      Red Troncal ALAMEDA (Santiago)-PUERTO (Valpara... -70.678887 -33.457018
...                                                  ...        ...        ...
96596                           Est ARICA (FFCC a Tacna) -70.317542 -18.473507
96597                                          Pte LLUTA -70.304477 -18.403892
96598                                     Est CHACALLUTA -70.325011 -18.318546
96599               Hito Límite Internacional Chile-Perú -70.325528 -18.313046
96600                                      Est ALPATACAL -67.248522 -33.554839

[96601 rows x 3 columns]


In [50]:
# Filtrar solo filas con "Est" en el nombre, excluyendo "Ex-Est"
df_estaciones_clean = df_stations[
    df_stations['name'].str.contains("Est", case=True, na=False) &
    ~df_stations['name'].str.contains("Ex-Est", case=True, na=False)
].copy()

# Eliminar duplicados por nombre y coordenadas (opcional)
df_estaciones_clean = df_estaciones_clean.drop_duplicates(subset=['name','longitude','latitude'])

# Resetear índice
df_estaciones_clean.reset_index(drop=True, inplace=True)

# Guardar CSV limpio
df_estaciones_clean.to_csv("estaciones_filtradas.csv", index=False)

print(df_estaciones_clean)


                         name  longitude   latitude
0       Est ALAMEDA (Central) -70.679052 -33.451823
1            Est YUNGAY NORTE -70.681263 -33.428082
2                   Est RENCA -70.690631 -33.400383
3               Est QUILICURA -70.705348 -33.364199
4                  Est COLINA -70.769486 -33.291047
..                        ...        ...        ...
762             Est LLAU-LLAU -73.767505 -42.462248
763                Est CASTRO -73.759805 -42.481223
764  Est ARICA (FFCC a Tacna) -70.317542 -18.473507
765            Est CHACALLUTA -70.325011 -18.318546
766             Est ALPATACAL -67.248522 -33.554839

[767 rows x 3 columns]


In [51]:
df_stations.to_csv("estaciones2.csv", index=False)
